# Simulating a system of Kern–Frenkel particles

## Overview

### Questions
* How do I implement pairwise energetic interactions in HPMC?

### Objectives
* Run a simulation of particles interacting through the Kern–Frenkel pair potential.
* Log the potential energy during an HPMC simulation.

## Boilerplate code

In [1]:
import gsd.hoomd
import hoomd
import itertools
import math
import matplotlib
import numpy

%matplotlib inline
matplotlib.style.use('ggplot')

In the previous section, you learned about the Kern–Frenkel model and saw a C++ code snippet that implements the model in a way that can be used in an HPMC simulation.
In this section, we will initialize a system of particles, add the Kern–Frenkel pair potential to the simulation, and run the simulation while logging the energy.

## Constructing the system
Construct the system using the same code you used in [Introducing HOOMD-blue tutorial](../00-Introducing-HOOMD-blue/03-Initializing-the-System-State.ipynb) and then initialize a simulation.

In [2]:
m = 4
N_particles = 2 * m**3
spacing = 1.2
K = math.ceil(N_particles**(1 / 3))
L = K * spacing
x = numpy.linspace(-L / 2, L / 2, K, endpoint=False)
position = list(itertools.product(x, repeat=3))
position = position[0:N_particles]
orientation = [(1, 0, 0, 0)] * N_particles

# gsd snapshot
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N_particles
snapshot.particles.position = position
snapshot.particles.orientation = orientation
snapshot.particles.typeid = [0] * N_particles
snapshot.particles.types = ['A']
snapshot.configuration.box = [L, L, L, 0, 0, 0]
with gsd.hoomd.open(name='initial.gsd', mode='xb') as f:
    f.append(snapshot)
    
# build simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=0)
sim.create_state_from_gsd(filename='initial.gsd')

### Add HPMC integrator
Now add an HPMC integrator to the system.
Since the cores of the particles are spherical, we add the `Sphere` integrator.
Since these particles also have an orientation, we must tell HPMC to perform rotation moves on the particles.

In [3]:
sigma = 1.0  # hard core diameter
kf_lambda = 1.2  # range of patchy interaction
kf_epsilon = 2.0  # strength of patchy interaction in kT
kf_delta_deg = 45  # half-opening angle of patches
mc = hoomd.hpmc.integrate.Sphere()
mc.shape['A'] = dict(diameter=sigma, orientable=True)
sim.operations.integrator = mc

### Add the pair potential
Now add the pair potential to the integrator.
Use the same code as in [the previous section](01-Kern-Frenkel.ipynb) with a slight formatting change.
Because we pass the code as a Python string, we must use double curly brackets in place of single curly brackets.
Set the 

In [4]:
patch_code = f"""
// constants
const float epsilon = {kf_epsilon:f};  // = 2 kT
const float sigma = {sigma:f};  // hard core radius
const float lambda = {kf_lambda:f};
const float delta = {kf_delta_deg} * M_PI / 180;  // delta in radians
const vec3<float> ehat_particle_reference(1, 0, 0);  // patch director, unit magnitude

// relevant quantities
ehat_i = rotate(q_i, ehat_particle_reference);
ehat_j = rotate(q_j, ehat_particle_reference);
float rsq = dot(r_ij, r_ij);
vec3<float> r_hat_ij = r_ij / sqrt(rsq);  // unit vector pointing from particle i to particle j
bool patch_on_i_is_aligned = dot(ehat_i, r_hat_ij) >= cos(delta);
bool patch_on_j_is_aligned = dot(ehat_j, -r_hat_ij) >= cos(delta);

// check for patch alignment and distance criterion
if (patch_on_i_is_aligned && patch_on_j_is_aligned && rsq < 2*lambda*sigma * 2*lambda*sigma)
    {{
    return -epsilon
    }}
else
    {{
    return 0.0;
    }}
"""
kf_potential = hoomd.hpmc.pair.user.CPPPotential(1.5, patch_code, [])
mc.pair_potential = kf_potential
sim.run(100)

RuntimeError: Error compiling JIT code:

                        #include <stdio.h>
                        #include "hoomd/HOOMDMath.h"
                        #include "hoomd/VectorMath.h"

                        // param_array (singlet class) or param_array_isotropic
                        // and param_array_constituent (union class) are
                        // allocated by the library
                        float *param_array;
                        

                        using namespace hoomd;

                        extern "C"
                        {
                        float eval(const vec3<float>& r_ij,
                            unsigned int type_i,
                            const quat<float>& q_i,
                            float d_i,
                            float charge_i,
                            unsigned int type_j,
                            const quat<float>& q_j,
                            float d_j,
                            float charge